In [1]:
from bs4 import BeautifulSoup as bs
import requests
import numpy as np
import pandas as pd

In [ ]:
base_url = "https://sofifa.com/players?r=210064&set=true&offset="
second_url = "https://sofifa.com"
offset = 0
columns = ['ID', 'Name', 'Age', 'Photo', 'Nationality', 'Flag', 'Overall', 'Potential', 'Club', 
           'Club Logo', 'Value', 'Wage', 'Total_stats', 'Height', 'Weight', 'Dribbling', 'Curve', 'FK Accuracy', \
           "Long Passing", "Ball Control", "Acceleration", "Sprint Speed", "Agility", "Reactions", "Balance", \
           "Shot Power","Jumping", "Stamina", "Strength", "Long Shots", "Aggression", "Interceptions", "Positioning", "Vision", \
          "Penalties", "Composure", "Defensive awareness", "Standing Tackle", "Sliding tackle", "GK Diving", "GK Handling", \
           "GK Kicking", "GK Positioning", "GK Reflexes"]

data = pd.DataFrame(columns=columns)
i=0
for offset in range(260, 290):
    url = base_url + str(offset*60)  # approx 60 players displayed on page
    source_code = requests.get(url)
    plain_text = source_code.text
    soup = bs(plain_text)
    table_body = soup.find('tbody')
    for row in table_body.findAll('tr'):
        td = row.findAll('td')
        picture = td[0].find('img').get('data-src')
        pid = td[0].find('img').get('id')
        name = td[1].findAll('a')[0].find('div').text
        nationality = td[1].find('img').get('title')
        flag_img = td[1].find('img').get('data-src')
        age = td[2].text.strip()
        overall = td[3].text.strip()
        potential = td[4].text.strip()
        club = td[5].find('a').text
        club_logo = td[5].find('img').get('data-src')
        value = td[6].text.strip().strip('€')
        wage = td[7].text.strip().strip('€')
        total_stats = td[8].text.strip()
        new_url = second_url + td[1].findAll('a')[0].get('href')
        more_info = requests.get(new_url)
        soup_2 = bs(more_info.text)
        weight=soup_2.find("div", attrs={"class":"meta ellipsis"}).text.split()[-1]
        height =soup_2.find("div", attrs={"class":"meta ellipsis"}).text.split()[-2]
        skills = []
        s = soup_2.findAll("div", attrs={"class":"center"})[5].findAll("div", attrs={"class":"block-quarter"})
        for j in range(1, 7):
            l = s[j].findAll('li')
            for m in l:
                skills.append(m.find('span').text)
        player_data = pd.DataFrame([[pid, name, age, picture, nationality, flag_img, overall, 
                                  potential, club, club_logo, value, wage, total_stats, height, weight]+skills])
        try:
            player_data.columns = columns
            data = data.append(player_data, ignore_index=True)
            i+=1
        except:
            continue
        print(i)
    offset+=1
    data.to_csv('player_info.csv',mode='a', encoding='utf-8', header=False, index=False)

In [17]:

positions=[['LS', 'ST', 'RS'],
['LW', 'LF', 'CF', 'RF', 'RW'],
 ['LAM', 'CAM', 'RAM'],
['LM', 'LCM', 'CM', 'RCM', 'RM'],
['LWB', 'LDM', 'CDM', 'RDM', 'RWB'],
['LB', 'LCB', 'CB', 'RCB', 'RB'],
['GK']]

columns=['ID', 'LS', 'ST', 'RS','LW', 'LF', 'CF', 'RF', 'RW','LAM', 'CAM',\
         'RAM','LM', 'LCM', 'CM', 'RCM', 'RM', 'LWB', 'LDM', 'CDM', 'RDM',\
         'RWB', 'LB', 'LCB', 'CB', 'RCB', 'RB', 'GK']
data = pd.DataFrame(columns=columns)
offset=0
second_url = "https://sofifa.com"
base_url = "https://sofifa.com/players?r=210064&set=true&offset="

final = pd.read_csv('football_final.csv')
url = 'https://sofifa.com/player/'

for i in final.loc[pd.isnull(final.LDM)]['ID'][:]:
    print(i)
    try:
        purl = url + str(i)  
        more_info = requests.get(purl)
        soup_2 = bs(more_info.text)
        all_positions = soup_2.find('div', attrs={'class':'field-small'}).findAll('div', attrs={'class':'grid half-spacing'})
        m = 0
        final_position = [i]
        for a in all_positions:
            for b in positions[m]:
                final_position.append(a.text[a.text.find(b)+len(b) : a.text.find(b)+len(b)+2])
            m+=1
        player_data = pd.DataFrame([final_position])
        player_data.columns = columns
        player_data.to_csv('position_data.csv',mode='a', encoding='utf-8', index=False, header=False, low_memory=False)
    except:
        continue

In [10]:
import pandas as pd
  
# reading csv files
data1 = pd.read_csv('player_info.csv', low_memory=False)
data2 = pd.read_csv('position_data.csv', low_memory=False)
output2 = pd.merge(data1, data2, 
                   on='ID', 
                   how='left')
output2.to_csv('football_final.csv', index=False)

In [14]:
data = pd.read_csv('football_final.csv', low_memory=False)

In [15]:
data

,ID,Name,Age,Photo,Nationality,Flag,Overall,Potential,Club,Club Logo,...,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,GK
0,2,16 G. Pasquale,33,https://cdn.sofifa.net/players/000/002/16_60.png,Italy,https://cdn.sofifa.net/flags/it.png,69,69,Udinese Calcio,https://cdn.sofifa.net/teams/55/30.png,...,70,70,70,71,70,69,69,69,70,17
1,241,14 R. Giggs,39,https://cdn.sofifa.net/players/000/241/14_60.png,Wales,https://cdn.sofifa.net/flags/gb-wls.png,78,78,Manchester United,https://cdn.sofifa.net/teams/11/30.png,...,63,63,63,62,57,53,53,53,57,11
2,965,11 N. Ventola,32,https://cdn.sofifa.net/players/000/965/11_60.png,Italy,https://cdn.sofifa.net/flags/it.png,62,80,Novara Calcio,https://cdn.sofifa.net/teams/112225/30.png,...,48,48,48,46,44,44,44,44,44,12
3,1179,G. Buffon,43,https://cdn.sofifa.net/players/001/179/22_60.png,Italy,https://cdn.sofifa.net/flags/it.png,80,80,Parma,https://cdn.sofifa.net/teams/50/30.png,...,32,32,32,28,26,28,28,28,26,78
4,1747,11 G. Coupet,37,https://cdn.sofifa.net/players/001/747/11_60.png,France,https://cdn.sofifa.net/flags/fr.png,75,83,Paris Saint-Germain,https://cdn.sofifa.net/teams/73/30.png,...,29,29,29,24,25,28,28,28,25,75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11935,244435,J. Heil,23,https://cdn.sofifa.net/players/244/435/21_60.png,Austria,https://cdn.sofifa.net/flags/at.png,65,70,TSV Egger Glas Hartberg,https://cdn.sofifa.net/teams/2017/30.png,...,65,65,65,64,63,64,64,64,63,15
11936,253139,S. Pierotti,19,https://cdn.sofifa.net/players/253/139/21_60.png,Argentina,https://cdn.sofifa.net/flags/ar.png,65,77,Club Atlético Colón,https://cdn.sofifa.net/teams/110406/30.png,...,65,65,65,65,64,63,63,63,64,17
11937,232916,K. Ogawa,27,https://cdn.sofifa.net/players/232/916/21_60.png,Japan,https://cdn.sofifa.net/flags/jp.png,65,65,Yokohama FC,https://cdn.sofifa.net/teams/113197/30.png,...,55,55,55,58,56,50,50,50,56,16
11938,236244,T. Sibbick,21,https://cdn.sofifa.net/players/236/244/21_60.png,England,https://cdn.sofifa.net/flags/gb-eng.png,65,73,Barnsley,https://cdn.sofifa.net/teams/1932/30.png,...,62,62,62,64,65,66,66,66,65,16
